In [1]:
%load_ext lab_black
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.formula.api as smf

In [2]:
potholes_1 = pd.read_csv("../data/potholes_1.csv")
potholes_2 = pd.read_csv("../data/potholes_2.csv")
census_data = pd.read_csv("../data/chicago_census_data.csv")

In [3]:
data = pd.concat([potholes_1, potholes_2], ignore_index=True)

In [4]:
# Renaming columns
data = data.rename(
    columns={
        "CREATION DATE": "Creation Date",
        "STATUS": "Status",
        "COMPLETION DATE": "Completion Date",
        "SERVICE REQUEST NUMBER": "Srn",
        "TYPE OF SERVICE REQUEST": "Type",
        "CURRENT ACTIVITY": "Current Activity",
        "MOST RECENT ACTION": "Action",
        "NUMBER OF POTHOLES FILLED ON BLOCK": "Potholes",
        "STREET ADDRESS": "Address",
        "ZIP": "zip",
        "X COORDINATE": "x coordinate",
        "Y COORDINATE": "y coordinate",
        "Ward": "ward",
        "Police District": "Police District",
        " Community Area": "Area",
        "SSA": "ssa",
        "LATITUDE": "Latitude",
        "LONGITUDE": "Longitude",
        "LOCATION": "Location",
    }
)
data.columns

Index(['Creation Date', 'Status', 'Completion Date', 'Srn', 'Type',
       'Current Activity', 'Action', 'Potholes', 'Address', 'zip',
       'x coordinate', 'y coordinate', 'ward', 'Police District',
       'Community Area', 'ssa', 'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [5]:
census_data = census_data.rename(
    columns={
        "Community Area Number": "Community Area",
        "PER CAPITA INCOME ": "PCI",
        "HARDSHIP INDEX": "HI",
    }
)
census_data["log_value_pci"] = np.log(census_data["PCI"])
census_data.columns

Index(['Community Area', 'COMMUNITY AREA NAME', 'PERCENT OF HOUSING CROWDED',
       'PERCENT HOUSEHOLDS BELOW POVERTY', 'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PCI', 'HI', 'log_value_pci'],
      dtype='object')

In [6]:
data_new = pd.merge(data, census_data, on=["Community Area"])
data_new.head()
data_new.shape

(558912, 28)

In [7]:
# setting date as an index
data_new["Creation Date"] = pd.to_datetime(data["Creation Date"])
data_new.set_index("Creation Date", inplace=True)

In [8]:
from datetime import datetime

a = datetime.strptime("2011-07-01", "%Y-%m-%d")
if a.month == datetime.now().month:
    print("This month")

In [9]:
# sorting by first name
data_new.sort_values("Srn", inplace=True)

# dropping ALL duplicate values
data_new.drop_duplicates(subset="Srn", keep=False, inplace=True)

In [10]:
# Replace the missing values for numerical columns with mean
data["ssa"] = data["ssa"].fillna(data["ssa"].mean())
data["Potholes"] = data["Potholes"].fillna(data["Potholes"].mean())
# Replace the missing values for numerical columns with mode
data["Current Activity"] = data["Current Activity"].fillna(
    data["Current Activity"].mode()[0]
)
data["Action"] = data["Action"].fillna(data["Action"].mode()[0])
data["x coordinate"] = data["x coordinate"].fillna(data["x coordinate"].mode()[0])
data["y coordinate"] = data["y coordinate"].fillna(data["y coordinate"].mode()[0])
# Replace the missing value of competion date by a future date using 'fillna' method
data["Completion Date"] = data["Completion Date"].fillna(12 / 18 / 2022)
data["zip"] = data["zip"].fillna(0)
data["Address"] = data["Address"].fillna(0)
data["ward"] = data["ward"].fillna(0)
data["Police District"] = data["Police District"].fillna(0)
data["Community Area"] = data["Community Area"].fillna(0)
data["Latitude"] = data["Latitude"].fillna(0)
data["Longitude"] = data["Longitude"].fillna(0)
data["Location"] = data["Location"].fillna(0)

In [11]:
census_data["Community Area"] = census_data["Community Area"].fillna(0)
census_data["HI"] = census_data["HI"].fillna(0)

In [12]:
census_data.isnull().sum()

Community Area                                  0
COMMUNITY AREA NAME                             0
PERCENT OF HOUSING CROWDED                      0
PERCENT HOUSEHOLDS BELOW POVERTY                0
PERCENT AGED 16+ UNEMPLOYED                     0
PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA    0
PERCENT AGED UNDER 18 OR OVER 64                0
PCI                                             0
HI                                              0
log_value_pci                                   0
dtype: int64

In [13]:
data.isnull().sum()

Creation Date       0
Status              0
Completion Date     0
Srn                 0
Type                0
Current Activity    0
Action              0
Potholes            0
Address             0
zip                 0
x coordinate        0
y coordinate        0
ward                0
Police District     0
Community Area      0
ssa                 0
Latitude            0
Longitude           0
Location            0
dtype: int64

In [14]:
results1 = smf.ols("Potholes ~ log_value_pci + HI", data=data_new).fit()
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:               Potholes   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1389.
Date:                Tue, 22 Mar 2022   Prob (F-statistic):               0.00
Time:                        15:48:51   Log-Likelihood:            -1.7026e+06
No. Observations:              395619   AIC:                         3.405e+06
Df Residuals:                  395616   BIC:                         3.405e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        39.9720      1.984     20.144

In [15]:
data.describe()

,Potholes,zip,x coordinate,y coordinate,ward,Police District,Community Area,ssa,Latitude,Longitude
count,560478.000000,560478.000000,5.604780e+05,5.604780e+05,560478.000000,560478.000000,560478.000000,560478.000000,560478.000000,560478.000000
mean,10.917616,60303.485999,1.160043e+06,1.892140e+06,25.707680,13.140468,35.124403,32.442022,41.793837,-87.550287
std,15.143209,4451.120310,1.736707e+04,3.548172e+04,14.222005,6.987750,23.516352,6.828806,1.662114,3.476376
min,0.000000,0.000000,1.092430e+06,1.813892e+06,0.000000,0.000000,0.000000,1.000000,0.000000,-87.936132
25%,2.000000,60619.000000,1.148706e+06,1.861000e+06,13.000000,8.000000,15.000000,32.442022,41.773565,-87.729681
50%,10.917616,60630.000000,1.160756e+06,1.898819e+06,26.000000,14.000000,29.000000,32.442022,41.877518,-87.685376
75%,10.917616,60643.000000,1.171910e+06,1.922107e+06,38.000000,18.000000,58.000000,32.442022,41.942314,-87.644831
max,320.000000,60827.000000,1.205133e+06,1.951593e+06,50.000000,31.000000,77.000000,69.000000,42.022960,0.000000


In [16]:
data_new.corr()

,Potholes,zip,x coordinate,y coordinate,ward,Police District,Community Area,ssa,Latitude,Longitude,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PCI,HI,log_value_pci
Potholes,1.000000,0.000006,0.037678,-0.099520,-0.070748,-0.057326,0.090452,0.033976,-0.049872,0.005556,0.008659,0.061410,0.101595,0.033857,0.104333,-0.080473,0.079662,-0.083554
zip,0.000006,1.000000,-0.010268,0.002492,0.009426,0.019625,0.001407,-0.020978,0.001598,-0.002359,-0.001181,-0.000829,0.001712,-0.002749,0.007109,-0.005658,-0.000150,-0.003348
x coordinate,0.037678,-0.010268,1.000000,-0.531707,-0.470308,-0.465859,0.287502,0.329935,-0.310696,0.215471,-0.216671,0.361744,0.251064,-0.170541,-0.086079,0.119437,0.058755,0.049353
y coordinate,-0.099520,0.002492,-0.531707,1.000000,0.717848,0.594048,-0.809395,-0.144119,0.578718,-0.111112,0.083402,-0.308073,-0.554050,-0.145511,-0.491409,0.334266,-0.389393,0.388310
ward,-0.070748,0.009426,-0.470308,0.717848,1.000000,0.666048,-0.587604,-0.179225,0.417538,-0.100754,-0.031336,-0.368981,-0.449368,-0.216895,-0.285876,0.286425,-0.359066,0.329905
Police District,-0.057326,0.019625,-0.465859,0.594048,0.666048,1.000000,-0.463177,-0.136880,0.346493,-0.100635,0.036177,-0.363221,-0.409937,-0.134631,-0.202985,0.210241,-0.293801,0.258477
Community Area,0.090452,0.001407,0.287502,-0.809395,-0.587604,-0.463177,1.000000,-0.013127,-0.467715,0.058639,0.008051,0.224738,0.486314,0.204934,0.458775,-0.417726,0.376242,-0.427209
ssa,0.033976,-0.020978,0.329935,-0.144119,-0.179225,-0.136880,-0.013127,1.000000,-0.144790,0.329725,-0.067587,0.170418,0.305178,-0.107203,0.224149,-0.147187,0.071013,-0.082955
Latitude,-0.049872,0.001598,-0.310696,0.578718,0.417538,0.346493,-0.467715,-0.144790,1.000000,-0.861236,0.050050,-0.178512,-0.318750,-0.081388,-0.279580,0.189592,-0.222184,0.220727
Longitude,0.005556,-0.002359,0.215471,-0.111112,-0.100754,-0.100635,0.058639,0.329725,-0.861236,1.000000,-0.048044,0.076695,0.049965,-0.040032,-0.025369,0.030929,0.007829,0.016236
